## 一、 第一个 Agents 
![第一个Agents LangChain中的Agents](/Users/baisongtao/mycode/LLMClass01/05-Agent/pngs/langchain_agent.png)

**任务设计**
- 会做数学题
- 不知道答案的时候可以搜索
____

In [20]:
from langchain_ollama.chat_models import ChatOllama

chat_llm = ChatOllama(model="qwen2.5:latest",temperature=0)

**搭建工具**
- serpai 是一个聚合搜索引擎，需要安装谷歌搜索包以及申请账号
- llm-math 是一个封装好的数学计算器

In [8]:
! pip install google-search-results

In [ ]:
# 这里没用谷歌的API，买了国内博查的API
import os
from dotenv import load_dotenv
load_dotenv()

# BOCHA_API_KEY = os.getenv("BOCHA_API_KEY")

In [10]:
from langchain.agents import load_tools

tools = load_tools(["llm-math"], llm=chat_llm)

In [28]:
# 定义Agent 使用小样本增强生成类型
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import Tool
from bocha_tool import BochaAPITool

bocha_tool = Tool(
      name="BochaWebSearch",
      func=BochaAPITool.bocha_websearch_tool,
      description="使用Bocha Web Search API 进行搜索互联网网页，输入应为搜索查询字符串，输出将返回搜索结果的详细信息，包括网页标题、网页URL、网页摘要、网站名称、网站Icon、网页发布时间等。"
  )

agent = initialize_agent(
  tools=[bocha_tool, *tools],
  llm=chat_llm,
  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # 这里有不同的类型
  verbose=True
)

In [29]:
agent.invoke("特朗普多少岁？他年纪的平方是多少？")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "(2023 - 1946) ** 2"
}
```
Observation: Answer: 5929
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "特朗普出生于1946年，到2023年的年龄是77岁。他年纪的平方是5929。"
}
```

> Finished chain.


{'input': '特朗普多少岁？他年纪的平方是多少？',
 'output': '特朗普出生于1946年，到2023年的年龄是77岁。他年纪的平方是5929。'}